In [1]:
!pip install pandas
!pip install py2neo


In [2]:
import pandas as pd
from py2neo import Graph, Node, Relationship

In [3]:
node_df = pd.read_csv('hetionet/nodes.tsv', sep='\t')
edges_df = pd.read_csv('hetionet/edges.tsv', sep='\t')

In [6]:
test = node_df.iloc[0]

if 'Anatomy' in test['id']:
    print('Anatomy')
else:
    print(0)

Anatomy


In [61]:
type_label, type_id = test['id'].split('::')
print(type_label)
print(type_id)

Anatomy
UBERON:0000002


Make connection to Neo4j database

In [17]:
graph = Graph("bolt://localhost:7687", auth=('neo4j', 'lucky238'))

Import node data into graph database

In [70]:
def create_Node(row):
    node_label, identifier = row['id'].split('::')

    if 'Anatomy' == node_label:
        anatomy_node = Node(label = node_label, id = identifier, name = row['name'])
        graph.merge(anatomy_node, 'Anatomy', 'id')
    
    elif 'Disease' == node_label:
        disease_node = Node(label = node_label, id= identifier, name = row['name'])
        graph.merge(disease_node, 'Disease', 'id')

    elif 'Compound' == node_label:
        compound_node = Node(label = node_label, id= identifier, name = row['name'])
        graph.merge(compound_node, 'Compound', 'id')
    
    elif 'Gene' == node_label:
        gene_node = Node(label = node_label, id = identifier, name = row['name'])
        graph.merge(gene_node, 'Gene', 'id')

In [71]:
node_df.apply(create_Node, axis=1)

0        None
1        None
2        None
3        None
4        None
         ... 
23031    None
23032    None
23033    None
23034    None
23035    None
Length: 23036, dtype: object

Import relationship data into graph database

In [72]:
edges_df['metaedge'].unique()

array(['GiG', 'CrC', 'DdG', 'DlA', 'CtD', 'CbG', 'CuG', 'DrD', 'DaG',
       'CpD', 'AdG', 'AuG', 'GcG', 'Gr>G', 'CdG', 'DuG', 'AeG'],
      dtype=object)

In [48]:
for index, row in edges_df.iterrows():
    source_id = row['source']
    target_id = row['target']
    rel_type = row['metaedge']

    source_node = graph.nodes.match('Disease', id=source_id).first()
    target_node = graph.nodes.match('Disease', id=target_id).first()

    if source_node and target_node:
        rel = Relationship(source_node, rel_type, target_node)

        graph.merge(rel)
    else:
        print("Cannot find")